In [11]:
import torch
from PIL import Image
from transformers import AutoImageProcessor, AutoModelForImageClassification
import pandas as pd
import os


In [12]:
images_dir = r"C:\Users\lucas\Documents\IPSSI\IA Cours\Projet\images"
class_folders = [name for name in os.listdir(images_dir) if os.path.isdir(os.path.join(images_dir, name))]

# Permet de nettoyer les noms en supprimant les 'n02..' devant le nom des dossiers
cleaned_names = [name.split('-')[-1] for name in class_folders]

# Créer un DataFrame avec la liste récup
df_classes = pd.DataFrame({
    "class_index": range(len(cleaned_names)),
    "class_name": cleaned_names
})

# Sauvegarder dans un fichier CSV si nécessaire 
df_classes.to_csv("class_mapping.csv", index=False)

#Check
print(df_classes.head())


   class_index        class_name
0            0         Chihuahua
1            1  Japanese_spaniel
2            2       Maltese_dog
3            3          Pekinese
4            4               Tzu


In [16]:
#On peut juste utiliser le df mais utilisation du csv ici pour le test 
mapping_path = r"C:\Users\lucas\Documents\IPSSI\IA Cours\Projet\class_mapping.csv"
df_classes = pd.read_csv(mapping_path)
class_names = df_classes.sort_values("class_index")["class_name"].tolist()

On récupère un modèle déjà préentrainé 

In [13]:
model_name = "anonauthors/stanford_dogs-resnet50"
processor = AutoImageProcessor.from_pretrained(model_name)
model = AutoModelForImageClassification.from_pretrained(model_name)

#Lien du modèle : https://huggingface.co/anonauthors/stanford_dogs-resnet50/blob/main/config.json

In [14]:
# Juste pour le test, à remplacer par un input plus tard quand on l'intégrera

img_path = r"C:\Users\lucas\Documents\IPSSI\IA Cours\Projet\images\n02112706-Brabancon_griffon\n02112706_16.jpg"
image = Image.open(img_path).convert("RGB")
inputs = processor(images=image, return_tensors="pt")

In [17]:
with torch.no_grad():
    outputs = model(**inputs)
    logits = outputs.logits
    idx = logits.argmax(-1).item()
    prob = torch.softmax(logits, dim=1)[0, idx].item()

pred_label = class_names[idx]
print(f"Classe prédite : **{pred_label}** (probabilité : {prob:.2f})")


Classe prédite : **Brabancon_griffon** (probabilité : 0.95)
